In [19]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Activation, RepeatVector, Dropout, Dense
from keras.layers import recurrent
from keras.layers.wrappers import TimeDistributed
import numpy as np

# Suppress the TensorFlow warning
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [20]:
# Function to convert integer input to vectors for RNN using one hot encoding

def encode(X,seq_len, vocab_size):
    x = np.zeros((len(X),seq_len, vocab_size), dtype=np.float32)
    for ind,batch in enumerate(X):
        for j, elem in enumerate(batch):
            x[ind, j, elem] = 1
    return x

In [21]:
# Generator function to generate infinite-stream of inputs for training

def batch_gen(batch_size=32, seq_len=10, max_no=100):
    # Randomly generate a batch of integer sequences (X) and its sorted counterpart (Y)
    x = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)
    y = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)

    while True:
	# Generates a batch of input
        X = np.random.randint(max_no, size=(batch_size, seq_len))

        Y = np.sort(X, axis=1)

        for ind,batch in enumerate(X):
            for j, elem in enumerate(batch):
                x[ind, j, elem] = 1

        for ind,batch in enumerate(Y):
            for j, elem in enumerate(batch):
                y[ind, j, elem] = 1

        yield x, y
        x.fill(0.0)
        y.fill(0.0)

In [23]:
RNN = recurrent.LSTM

# Global parameters.
batch_size=32
seq_len = 10
max_no = 100

model = Sequential()
# the encoder LSTM
model.add(LSTM(hidden_size, input_shape=(seq_len, max_no)))
# in next layer, repeat the input seq_len times
model.add(RepeatVector(seq_len))
# decoder RNN, which will return output sequence
for _ in range(n_layers):
    model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(max_no)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# Now the training loop, we'll sample input batches from the generator function written previously and feed it to the RNN for learning

for ind,(X,Y) in enumerate(batch_gen(batch_size, seq_len, max_no)):
    loss, acc = model.train_on_batch(X, Y)
    # We'll test RNN after each 250 iteration to check how well it is performing
    if ind % 250 == 0:
        testX = np.random.randint(max_no, size=(1, seq_len))
        test = encode(testX, seq_len, max_no)
        print testX
        #pdb.set_trace()
        y = model.predict(test, batch_size=1)
        print "actual sorted output is"
        print np.sort(testX)
        print "sorting done by RNN is"
        print np.argmax(y, axis=2)
        print "\n"

[[33 49 25  2 39  0  7 51 15 14]]
actual sorted output is
[[ 0  2  7 14 15 25 33 39 49 51]]
sorting done by RNN is
[[94 94 94 94 94 94 94 94 94 94]]


[[59 99  4 76 18 98 69 59 39 25]]
actual sorted output is
[[ 4 18 25 39 59 59 69 76 98 99]]
sorting done by RNN is
[[ 2 12 25 36 52 52 72 79 91 91]]


[[59 59 87 72 68 60 44 38 86 74]]
actual sorted output is
[[38 44 59 59 60 68 72 74 86 87]]
sorting done by RNN is
[[27 44 51 59 66 66 74 74 84 90]]


[[82 61 78 61 53 64 65 59  8 88]]
actual sorted output is
[[ 8 53 59 61 61 64 65 78 82 88]]
sorting done by RNN is
[[ 2 52 55 61 61 61 65 75 81 93]]


[[48 81 68 72  7 73 11 75 61 93]]
actual sorted output is
[[ 7 11 48 61 68 72 73 75 81 93]]
sorting done by RNN is
[[ 1 15 45 61 68 73 75 78 81 93]]


[[63 42  3  0 54 12 11 37  2 61]]
actual sorted output is
[[ 0  2  3 11 12 37 42 54 61 63]]
sorting done by RNN is
[[ 0  5  5  8 14 37 42 54 63 63]]


[[78 51 76 73 22 55 80 34 97 63]]
actual sorted output is
[[22 34 51 55 63 73 76 78 80 97]]
so

[[16 29 68 87 88 43  7 46 54  1]]
actual sorted output is
[[ 1  7 16 29 43 46 54 68 87 88]]
sorting done by RNN is
[[ 1  7 16 29 43 46 54 68 87 88]]


[[23 71 12 16 66 36  0 90 87 45]]
actual sorted output is
[[ 0 12 16 23 36 45 66 71 87 90]]
sorting done by RNN is
[[ 0 12 16 23 36 45 66 71 87 90]]


[[33 42 67  0 27  9 88 85 48 91]]
actual sorted output is
[[ 0  9 27 33 42 48 67 85 88 91]]
sorting done by RNN is
[[ 0  9 26 33 42 48 67 85 88 91]]


[[31 80 12 52  9 52 87 99 54 90]]
actual sorted output is
[[ 9 12 31 52 52 54 80 87 90 99]]
sorting done by RNN is
[[ 9 12 31 52 52 54 80 87 90 99]]


[[34 27 13 81 45 57 53 63 48 12]]
actual sorted output is
[[12 13 27 34 45 48 53 57 63 81]]
sorting done by RNN is
[[12 13 27 34 45 48 53 57 63 81]]


[[88 70 61 12 45 97 41 91 63 77]]
actual sorted output is
[[12 41 45 61 63 70 77 88 91 97]]
sorting done by RNN is
[[12 41 45 61 63 70 77 88 91 97]]


[[97 62 64 85 33 58 60 63 55 15]]
actual sorted output is
[[15 33 55 58 60 62 63 64 85 97]]
so

KeyboardInterrupt: 